In [1]:
import datetime
import warnings

import ipywidgets as widgets

%load_ext autoreload
%autoreload 2

from content import Correlations, Drawdown, RebasedStockGraph, StockStatistics
from controller import Controller
from data import FfnDataLoader
from gui import Gui
from user_input import UserInputWidget

warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
stock_lookup = {
    "SNP500": "spy",
    "Apple": "aapl",
    "Microsoft": "msft",
    "Facebook": "fb",
    "Tesla": "tsla",
    "Google": "googl",
}

<IPython.core.display.Javascript object>

In [4]:
### User input area ###
companies = UserInputWidget(
    "companies",
    widgets.SelectMultiple(
        options=stock_lookup.keys(),
        description="Stock",
        value=list(stock_lookup.keys())[:2],  # select first two
        disabled=False,
        layout=widgets.Layout(height="150px"),
    ),
)

start_date = UserInputWidget(
    "start_date",
    widgets.DatePicker(
        description="Start Date",
        value=datetime.datetime.today() - datetime.timedelta(days=3 * 365),
    ),
)

end_date = UserInputWidget(
    "end_date",
    widgets.DatePicker(description="End Date", value=datetime.datetime.today()),
)

get_stock_data_button = widgets.Button(
    description="Get data for selected stocks",
    disabled=False,
    button_style="info",
    tooltip="Click me",
    layout=widgets.Layout(width="auto", height="30px", margin="30px"),
)

<IPython.core.display.Javascript object>

In [5]:
## main chart
rebased_stock_graph = RebasedStockGraph("Rebased", "Rebased share price performance")


## tabs
stock_statistics = StockStatistics("Stock Statistics", "Statistics for each stock")
correlations = Correlations("Correlations", "Correlations between selected stocks")
drawdown = Drawdown("Drawdown", "Drawdown periods")

<IPython.core.display.Javascript object>

In [6]:
### initialise GUI (graphical user interface)
gui = Gui(
    user_inputs=[companies, start_date, end_date],
    action_button=get_stock_data_button,
    main=rebased_stock_graph,
    tabs=[stock_statistics, correlations, drawdown],
)

<IPython.core.display.Javascript object>

# Stock Price Comparison Tool

### Select multiple companies to compare their stock performance
 
<mark>Use ctrl+click to select multiple companies</mark>

<!-- TODO:
- Add linked histogram to select part of the graph and display the returns profile
- Tidy up css
- Add tab with optimial portfolio calculation
- Improve chart layouts
-->

In [7]:
### run application
Controller(gui, data_loader=FfnDataLoader(), stock_lookup=stock_lookup)

<IPython.core.display.Javascript object>

Created by [EngineeringforDataScience](https://engineeringfordatascience.com/) ⚒️ | [Source Code](https://github.com/julian-west/e4ds-snippets/tree/voila/jupyter/voila)

In [8]:
# Portfolio optimisation

<IPython.core.display.Javascript object>